<a href="https://colab.research.google.com/github/sravyapitani/FMMLlab/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [149]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [150]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-150-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [151]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [152]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [153]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [154]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [155]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [156]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [157]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [158]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [159]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [160]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

**ANSWER OF QUESTION 1:**

* When we increase the percentage of the validation set, the accuracy of the validation set might increase because we have more data to evaluate the model's performance. This can help in detecting overfitting or underfitting.

* If we reduce the percentage of the validation set, the accuracy might decrease because we have less data to evaluate the model's performance. This can lead to less reliable estimates of the model's generalization ability.

* Example code is given below:

In [161]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the Iris dataset as an example
data = load_iris()
X, y = data.data, data.target

# Vary the validation set percentage
validation_percentages = [56, 65, 95]  # Try different percentages

for val_percentage in validation_percentages:
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_percentage, random_state=42)

    # Create and train a model (Logistic Regression as an example)
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = model.predict(X_val)

    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)

    print(f"Validation Percentage: {val_percentage * 100}%")
    print(f"Accuracy: {accuracy:.2f}\n")


Validation Percentage: 5600%
Accuracy: 1.00

Validation Percentage: 6500%
Accuracy: 1.00

Validation Percentage: 9500%
Accuracy: 0.97



**ANSWER OF QUESTION 2:**

* The size of the train and validation sets can affect how well we can predict the accuracy on the test set using the validation set.

* If we have a larger training set, it can help the model learn more patterns and improve its performance. This can potentially lead to better predictions on the test set.

* Similarly if we have a larger validation set, it provides more representative sample of the data for evaluating the models performance. This can give us a better estimate of how well the model will generalize to unseen data, including the test set.

* So, having larger train and validation sets can generally lead to more accurate predictions on the test set.

* Example code is given below:

In [162]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming you have your data and labels stored in X and y variables

# Vary the train and validation set sizes
train_sizes = [0.6, 0.7, 0.8, 0.9]  # Train set sizes as a fraction of the total dataset

for train_size in train_sizes:
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)

    # Further split the train set into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=42)
    # Train the model on the train set
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Predict on the validation set
    val_predictions = model.predict(X_val)

    # Predict on the test set
    test_predictions = model.predict(X_test)

    # Calculate and print the accuracy scores
    val_accuracy = accuracy_score(y_val, val_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    print(f"Train size: {train_size}, Validation accuracy: {val_accuracy}, Test accuracy: {test_accuracy}")



Train size: 0.6, Validation accuracy: 0.8888888888888888, Test accuracy: 0.9833333333333333
Train size: 0.7, Validation accuracy: 0.9523809523809523, Test accuracy: 1.0
Train size: 0.8, Validation accuracy: 0.9583333333333334, Test accuracy: 1.0
Train size: 0.9, Validation accuracy: 1.0, Test accuracy: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**ANSWER OF QUESTION 3:**

* It's generally recommended to reverse around 20-30% of the dataset for the validation set. This allows for a good balance between having enough data for training the model and having a sufficient sample for evaluating its performance.

* However, the ideal percentage can vary depending on the size of the dataset and the complexity of the problem.

* It is a good practice to experiment with different percentages and observe the model's performance on the validation set to find the optimal balance.

* Example code is given below:

In [163]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming you have your data and labels stored in X and y variables

# Split the data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)

# Train the model on the train set
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the validation set
val_predictions = model.predict(X_val)

# Predict on the test set
test_predictions = model.predict(X_test)

# Calculate and print the accuracy scores
val_accuracy = accuracy_score(y_val, val_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Validation accuracy: {val_accuracy}, Test accuracy: {test_accuracy}")


Validation accuracy: 1.0, Test accuracy: 1.0


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [164]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [165]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**ANSWER OF FIRST QUESTION 1:**

* Yes, averaging the validation accuracy across multiple splits can help provide more consistent results.By performing multiple splits of the data into training and validation sets and calculating the accuracy for each split, we can reduce the impact of the specific data distribution in a single split.

* Averaging the accuracy across these splits give us a more reliable estimate of the models performance.

In [186]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Assuming you have your data and labels stored in X and y variables

# Create a logistic regression model
model = LogisticRegression()

# Perform cross-validation with 5 folds
scores = cross_val_score(model, X, y, cv=5)

# Calculate the average validation accuracy
average_accuracy = scores.mean()

print(f"Average validation accuracy: {average_accuracy}")


Average validation accuracy: 0.9733333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**ANSWER OF QUESTION 2:**

* Yes, averaging the validation accuracy across multiple splits can give a more accurate estimate of the test accuracy.
* By using multiple splits, we can reduce the impact of any specific data distribution or random variation in a single split.
* This helps to provide a more reliable and robust assessment of how well the model will perform on unseen data, such as the test set.  

In [187]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Assuming you have your data and labels stored in X and y variables

# Create a logistic regression model
model = LogisticRegression()

# Perform cross-validation with 5 folds
scores = cross_val_score(model, X, y, cv=5)

# Calculate the average test accuracy
average_accuracy = scores.mean()

print(f"Average test accuracy: {average_accuracy}")


Average test accuracy: 0.9733333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**ANSWER OF QUESTION 3:**

* Yes, increasing the number of iterations can potentially lead to a better estimate of the model's performance.

* With more iterations, the model has more opportunities to learn and adjust its parameters, which can improve its ability to generalize to unseen data.

* However, it's important to find the right balance. Increasing the number of iterations too much can lead to overfitting, where the model becomes too specialized to the training data and performs poorly on new data.

* It's crucial to monitor the model's performance on the validation set and choose the optimal number of iterations that maximizes accuracy without overfitting.

In [188]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Assuming you have your data and labels stored in X and y variables

# Create a logistic regression model with different number of iterations
model_100 = LogisticRegression(max_iter=100)
model_1000 = LogisticRegression(max_iter=1000)

# Perform cross-validation with 5 folds
scores_100 = cross_val_score(model_100, X, y, cv=5)
scores_1000 = cross_val_score(model_1000, X, y, cv=5)

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Assuming you have your data and labels stored in X and y variables

# Create a logistic regression model with different number of iterations
model_100 = LogisticRegression(max_iter=100)
model_1000 = LogisticRegression(max_iter=1000)

# Perform cross-validation with 5 folds
scores_100 = cross_val_score(model_100, X, y, cv=5)
scores_1000 = cross_val_score(model_1000, X, y, cv=5)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

**ANSWER OF QUESTION 4:**
* Yes, increasing the iterations can help mitigate the impact of a small training or validation dataset to some extent. With more iterations, the model has more chances to learn and adjust its parameters, potentially improving its performance.
* However, it's important to note that increasing iterations alone may not fully compensate for the limitations of a small dataset. It's still crucial to have a sufficient amount of diverse and representative data for the model to learn effectively.
* Additionally, other techniques like data augmentation or transfer learning can also be helpful in improving model performance with limited data.

In [189]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Assuming you have your data and labels stored in X and y variables

# Create a logistic regression model with different number of iterations
model_100 = LogisticRegression(max_iter=100)
model_1000 = LogisticRegression(max_iter=1000)

# Perform cross-validation with 5 folds
scores_100 = cross_val_score(model_100, X, y, cv=5)
scores_1000 = cross_val_score(model_1000, X, y, cv=5)

# Calculate the average test accuracies
average_accuracy_100 = scores_100.mean()
average_accuracy_1000 = scores_1000.mean()

print(f"Average test accuracy with 100 iterations: {average_accuracy_100}")
print(f"Average test accuracy with 1000 iterations: {average_accuracy_1000}")


Average test accuracy with 100 iterations: 0.9733333333333334
Average test accuracy with 1000 iterations: 0.9733333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
